In [41]:
import os

In [42]:
%pwd

'e:\\endtoendpost'

In [43]:
os.chdir("../")

In [44]:
%pwd

'e:\\'

In [45]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/vinayakavirat008/post-predictions-.mlflow "
os.environ["MLFLOW_TRACKING_USERNAME"]="vinayakavirat008"
os.environ["MLFLOW_TRACKING_PASSWORD"]="8006a8c9e77082e90d96573a13bb278110094dff"

In [46]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [47]:
from source.postpredictions import *


In [48]:

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/vinayakavirat008/post-predictions-.mlflow"
        )

        return model_evaluation_config


In [2]:
import os
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from source.postpredictions.utils.utils import load_object
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from fastapi import FastAPI

In [61]:
class ModelEvaluation:
    def __init__(self):
        pass
    
    def log_info(self, message):
        mlflow.log_text("info", message)

    def initiate_model_evaluation(self, train_array, test_array):
        try:
            # Assuming 'Emotion' is the target column
            target_column = 'Emotion'

            # Check if the target column is present in the DataFrame
            if target_column not in test_array.columns:
                raise ValueError(f"The target column '{target_column}' is not present in the DataFrame.")

            # Split features (X_test) and target variable (y_test) from the test_array
            X_test = test_array.drop(columns=[target_column])
            y_test = test_array[target_column]

            # Load the trained model
            model_path = os.path.join("artifacts", "model.pkl")
            model = load_object(model_path)

            # Set the MLflow registry URI
            mlflow.set_registry_uri("https://dagshub.com/vinayakavirat008/post-predictions-.mlflow")
            
            # Get the type of the tracking URI
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            
            self.log_info(f"Tracking URL Type: {tracking_url_type_store}")

            # Start a new MLflow run
            with mlflow.start_run():
                print(mlflow.get_tracking_uri())

                self.log_info("Making predictions using the test set")
                predicted_qualities = model.predict(X_test)

                self.log_info("Evaluating the model and logging classification metrics")
                accuracy = accuracy_score(y_test, predicted_qualities)
                classification_rep = classification_report(y_test, predicted_qualities)
                confusion_mat = confusion_matrix(y_test, predicted_qualities)

                # Log metrics
                mlflow.log_metric("accuracy", accuracy)

                # If using a remote registry, register the model in the Model Registry
                if tracking_url_type_store != "file":
                    mlflow.sklearn.log_model(model, "model", registered_model_name="ml_model")
                else:
                    # If using a local file store, log the model without registering
                    mlflow.sklearn.log_model(model, "model")

                # Log classification metrics
                mlflow.log_text("classification_report", classification_rep)
                mlflow.log_artifact(confusion_mat, "confusion_matrix.txt")
        
        except Exception as e:
            self.log_info(f"Exception occurred: {str(e)}")
            raise e